# Project
- 70% des utilisateurs souhaitent plus d'informations à propos de leur destination
- Chercher les meilleures villes en fonction de la météo 
- Proposer des hôtels sur ces villes

In [1]:
import requests
import json
import pandas as pd
import os
from dotenv import load_dotenv  

# Récupération des coordonnées GPS des 35 meilleures villes où voyager en France selon One Week In.com

### Test de l'API sur une ville pour voir si l'API renvoie bien quelque chose

In [2]:
url = 'https://data.geopf.fr/geocodage/search'

In [3]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3 ',
    "Accept-Language": "fr-FR,fr;q=0.9"
}

payload = {
    'q': 'Mont Saint Michel', 'France'
    'city': 'Mont Saint Michel',
    'format': 'json'
}

response = requests.get(url=url, params=payload, headers=headers)
response.json()


{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [5.995538, 45.546586]},
   'properties': {'label': 'Mont Saint Michel 73190 Challes-les-Eaux',
    'score': 0.946500909090909,
    'type': 'locality',
    'importance': 0.41151,
    'id': '73064_B039',
    'banId': 'f67c5810-39e4-4a3d-a92c-89380b8e6ab3',
    'name': 'Mont Saint Michel',
    'postcode': '73190',
    'citycode': '73064',
    'x': 933687.63,
    'y': 6498554.9,
    'city': 'Challes-les-Eaux',
    'context': '73, Savoie, Auvergne-Rhône-Alpes',
    'locality': 'Mont Saint Michel',
    '_type': 'address'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-1.538198, 47.243963]},
   'properties': {'label': 'Avenue du Mont Saint Michel 44300 Nantes',
    'score': 0.6999590909090908,
    'id': '44109_5785',
    'banId': 'bfc8760b-6503-436a-b194-96398ee4dd18',
    'name': 'Avenue du Mont Saint Michel',
    'postcode': '44300',
    'citycode': '441

### L'API répond, du coup on extrait les coordonnées GPS des 35 villes 

In [4]:
# Liste des villes à interroger
villes = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]



base_url = "https://data.geopf.fr/geocodage/search?"


def get_coordinates(ville):
    
    params = {
        'q': ville,
        'limit': 1,
        'index':'poi'
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data['features']:
            coordinates = data['features'][0]['geometry']['coordinates']
            return coordinates
        else:
            return None
    else:
        print(f"Erreur lors de la requête pour {ville}: {response.status_code}")
        return None


results = []
for ville in villes:
    coordinates = get_coordinates(ville)
    if coordinates:
        results.append({'Ville': ville, 'Longitude': coordinates[0], 'Latitude': coordinates[1]})
    else:
        results.append({'Ville': ville, 'Longitude': None, 'Latitude': None})


In [5]:
response.json()['features'][0]['geometry']['coordinates'] #recherche du chemin pour extraire les coordonnées GPS

[5.995538, 45.546586]

### On met les résultats obtenus dans un dataframe

In [ ]:
df = pd.DataFrame(results) # Nous avons bien un dataframe qui contient les 35 villes et leurs coordonnées GPS
df.head()

,Ville,Longitude,Latitude
0,Mont Saint Michel,7.088900,43.849841
1,St Malo,-2.000385,48.653039
2,Bayeux,-0.702368,49.277173
3,Le Havre,0.128407,49.493515
4,Rouen,1.099092,49.443440


In [8]:
df.shape # verification des dimensions savoir si on a bien 35 villes 

(35, 3)

In [9]:
df.isnull().any() #verification des valeurs nulles

Ville        False
Longitude    False
Latitude     False
dtype: bool

### On double check voir si toutes nos coordonnées GPS se situent bien en France

In [11]:
import plotly.express as px
fig = px.scatter_mapbox(df, lat="Latitude", lon="Longitude", text="Ville",
                        zoom=4, 
                        mapbox_style="open-street-map",
                        title="Verification si mes coordonnées reçues sont bien en France")
fig.update_traces(marker=dict(size=12))
fig.show()

# Maintenant que l'on a les coordonnées GPS pour nos 35 villes on va interroger une autre API pour avoir la météo, notamment la température et les précipications en mm de pluie

On fait un test de l'API sur une ville voir si l'API répond

In [124]:
base_weather_url = 'https://api.open-meteo.com/v1/forecast'
load_dotenv()  # Charger les variables d'environnement depuis le fichier .env


def get_weather(lat, lon):
    #appid = os.getenv('APPID')  # Récupérer l'API key depuis les variables d'environnement
    params = {
        'latitude': lat,
        'longitude': lon,
        'timezone': "Europe/Berlin",
        'timezone_abbreviation': "CEST",
        'daily': ["temperature_2m_max", "temperature_2m_mean", "precipitation_hours"]
    }
    
    response = requests.get(base_weather_url, params=params)
    print("Status code:", response.status_code)
    print("URL:", response.url)
    print("Response:", response.text)
    return response.json() if response.status_code == 200 else None



In [125]:
# Test with the first city
test_city = df.iloc[1]
weather_data = get_weather(test_city['Longitude'], test_city['Latitude'])
print(weather_data)

Status code: 200
URL: https://api.open-meteo.com/v1/forecast?latitude=-2.000385&longitude=48.653039&timezone=Europe%2FBerlin&timezone_abbreviation=CEST&daily=temperature_2m_max&daily=temperature_2m_mean&daily=precipitation_hours
Response: {"latitude":-2.0,"longitude":48.625,"generationtime_ms":0.45311450958251953,"utc_offset_seconds":7200,"timezone":"Europe/Berlin","timezone_abbreviation":"GMT+2","elevation":0.0,"daily_units":{"time":"iso8601","temperature_2m_max":"°C","temperature_2m_mean":"°C","precipitation_hours":"h"},"daily":{"time":["2025-05-02","2025-05-03","2025-05-04","2025-05-05","2025-05-06","2025-05-07","2025-05-08"],"temperature_2m_max":[29.6,29.5,29.6,29.2,29.0,29.9,30.0],"temperature_2m_mean":[29.2,29.1,29.1,29.0,28.6,29.1,29.3],"precipitation_hours":[10.0,22.0,11.0,17.0,24.0,9.0,15.0]}}
{'latitude': -2.0, 'longitude': 48.625, 'generationtime_ms': 0.45311450958251953, 'utc_offset_seconds': 7200, 'timezone': 'Europe/Berlin', 'timezone_abbreviation': 'GMT+2', 'elevation': 

L'API répond, du coup on peut faire la même démarche pour nos 35 villes.\
On boucle sur nos 35 villes souhaitées et on met nos données dans un dataframe pour plus de visibilité

In [126]:
def json_to_df(weather_data, city, lat, lon):
    if weather_data and "daily" in weather_data:
        df = pd.DataFrame(weather_data["daily"])
        df["city"] = city
        df["latitude"] = lat
        df["longitude"] = lon
        return df
    else:
        return pd.DataFrame()  # DataFrame vide

# Génération du DataFrame final
df_final = pd.DataFrame()

for city in villes:
    lat, lon = get_coordinates(city)
    if lat is not None and lon is not None:
        weather_data = get_weather(lat, lon)
        df_city = json_to_df(weather_data, city, lat, lon)
        df_final = pd.concat([df_final, df_city], ignore_index=True)
        print(f"{city} OK ({lat}, {lon})")
        

df_final.reset_index(drop=True, inplace=True)
df_final.head()

Status code: 200
URL: https://api.open-meteo.com/v1/forecast?latitude=7.0889&longitude=43.849841&timezone=Europe%2FBerlin&timezone_abbreviation=CEST&daily=temperature_2m_max&daily=temperature_2m_mean&daily=precipitation_hours
Response: {"latitude":7.125,"longitude":43.875,"generationtime_ms":0.03540515899658203,"utc_offset_seconds":7200,"timezone":"Europe/Berlin","timezone_abbreviation":"GMT+2","elevation":900.0,"daily_units":{"time":"iso8601","temperature_2m_max":"°C","temperature_2m_mean":"°C","precipitation_hours":"h"},"daily":{"time":["2025-05-02","2025-05-03","2025-05-04","2025-05-05","2025-05-06","2025-05-07","2025-05-08"],"temperature_2m_max":[30.7,28.9,30.3,28.0,28.8,28.5,30.7],"temperature_2m_mean":[25.3,24.6,25.3,24.2,24.7,24.7,25.8],"precipitation_hours":[7.0,7.0,7.0,10.0,9.0,12.0,3.0]}}
Mont Saint Michel OK (7.0889, 43.849841)
Status code: 200
URL: https://api.open-meteo.com/v1/forecast?latitude=-2.000385&longitude=48.653039&timezone=Europe%2FBerlin&timezone_abbreviation=CE

,time,temperature_2m_max,temperature_2m_mean,precipitation_hours,city,latitude,longitude
0,2025-05-02,30.7,25.3,7.0,Mont Saint Michel,7.0889,43.849841
1,2025-05-03,28.9,24.6,7.0,Mont Saint Michel,7.0889,43.849841
2,2025-05-04,30.3,25.3,7.0,Mont Saint Michel,7.0889,43.849841
3,2025-05-05,28.0,24.2,10.0,Mont Saint Michel,7.0889,43.849841
4,2025-05-06,28.8,24.7,9.0,Mont Saint Michel,7.0889,43.849841


In [ ]:
df_final.shape # on vérifie les dimensions du dataframe final. On a 245 lignes car nous avons 35 villes avec 7 jours de prévisions météo par ville

(245, 7)

In [128]:
df_final.isnull().any() #verification des valeurs nulles

time                   False
temperature_2m_max     False
temperature_2m_mean    False
precipitation_hours    False
city                   False
latitude               False
longitude              False
dtype: bool

In [129]:
df_final['city'].nunique() # verification rapide du nombre de ville unique dans notre dataframe

35

In [130]:
# On renomme les colonnes pour plus de clarté
df_final.rename(columns={
    'temperature_2m_max': 'Temperature_max (°C)',
    'temperature_2m_mean': 'Temperature_moyenne',
    'precipitation_hours': 'Temps_pluie (h)',
    'city': 'Ville',
    'time': 'Date'
}, inplace=True)

df_final.head()

,Date,Temperature_max (°C),Temperature_moyenne,Temps_pluie (h),Ville,latitude,longitude
0,2025-05-02,30.7,25.3,7.0,Mont Saint Michel,7.0889,43.849841
1,2025-05-03,28.9,24.6,7.0,Mont Saint Michel,7.0889,43.849841
2,2025-05-04,30.3,25.3,7.0,Mont Saint Michel,7.0889,43.849841
3,2025-05-05,28.0,24.2,10.0,Mont Saint Michel,7.0889,43.849841
4,2025-05-06,28.8,24.7,9.0,Mont Saint Michel,7.0889,43.849841


In [131]:
df_final.dtypes # on vérifie que la colonne date est bien au format datetime

Date                     object
Temperature_max (°C)    float64
Temperature_moyenne     float64
Temps_pluie (h)         float64
Ville                    object
latitude                float64
longitude               float64
dtype: object

In [132]:
# Elle est au format object, donc on va donc la convertir

df_final['Date'] = pd.to_datetime(df_final['Date'], format='%Y-%m-%d')
df_final.dtypes

Date                    datetime64[ns]
Temperature_max (°C)           float64
Temperature_moyenne            float64
Temps_pluie (h)                float64
Ville                           object
latitude                       float64
longitude                      float64
dtype: object

In [133]:
df_final.head()

,Date,Temperature_max (°C),Temperature_moyenne,Temps_pluie (h),Ville,latitude,longitude
0,2025-05-02,30.7,25.3,7.0,Mont Saint Michel,7.0889,43.849841
1,2025-05-03,28.9,24.6,7.0,Mont Saint Michel,7.0889,43.849841
2,2025-05-04,30.3,25.3,7.0,Mont Saint Michel,7.0889,43.849841
3,2025-05-05,28.0,24.2,10.0,Mont Saint Michel,7.0889,43.849841
4,2025-05-06,28.8,24.7,9.0,Mont Saint Michel,7.0889,43.849841


# A ENREGISTRER EN CSV

On choisit les 5 meilleures destinations en fonction des températures moyennes et precipitations moyennes

In [135]:
# on va chercher les 5 villes avec la température moyenne la plus élevée sur la période de 7 jours ET un temps de pluie inférieur à 1h
# On filtre le DataFrame pour ne garder que les heures avec peu ou pas de pluie
df_filtre = df_final[df_final["Temps_pluie (h)"] < 1]

# Puis on fait la moyenne des températures pour chaque ville, et on trie
top5 = df_filtre.groupby('Ville')['Temperature_moyenne'].mean().sort_values(ascending=False).head(5)

print(top5)


Ville
Lille        29.7
Amiens       29.6
Besancon     29.4
Collioure    29.4
Rouen        29.2
Name: Temperature_moyenne, dtype: float64


Pour la suite de notre projet, et notamment du scrapping des hôtels sur Booking, nous nous concentrerons sur les villes ci-dessus, choisies en fonction de leur température moyenne sur 7 jours ainsi que du temps de pluie journalier inférieur à 1h